In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.4.1
Num GPUs Available:  1


In [2]:
import pathlib
data_dir = pathlib.Path(r'path')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

58587


In [3]:
batch_size = 10
img_height = 85
img_width = 85

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=123,
    interpolation="bicubic",
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=123,
    interpolation="bicubic",
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

Found 58587 files belonging to 4 classes.
Using 41011 files for training.
Found 58587 files belonging to 4 classes.
Using 17576 files for validation.
['L', 'M', 'N', 'R']


In [4]:
from tensorflow.keras import layers
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255,input_shape=(img_height, img_width, 3)),
    layers.experimental.preprocessing.RandomContrast(0.1),
    layers.Conv2D(6, (3,3),strides=1, activation='relu',use_bias=True),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(8, (3,3), activation='relu',use_bias=True),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(14, (3,3), activation='relu',use_bias=True),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dropout(0.1),
    layers.Dense(40, activation='relu'),
    layers.Dense(40, activation='relu'),
    layers.Dense(4, activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0003)
model.compile(
  optimizer = optimizer,
  loss = 'sparse_categorical_crossentropy',
  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 85, 85, 3)         0         
_________________________________________________________________
random_contrast (RandomContr (None, 85, 85, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 83, 83, 6)         168       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 41, 41, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 39, 8)         440       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 14)        1

In [5]:
path = r'path\logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=1)

checkpoint_filepath = r'path\checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [6]:
from IPython.core.display import display, HTML
history = model.fit(
  train_ds,
  validation_data=val_ds,
  batch_size=500,
  epochs=11,
  callbacks=[model_checkpoint_callback,tensorboard_callback]
)

Epoch 1/11
4102/4102 [==============================] - 69s 16ms/step - loss: 0.9222 - accuracy: 0.6162 - val_loss: 0.4962 - val_accuracy: 0.8071
INFO:tensorflow:Assets written to: C:\Users\georg\Desktop\projects\Bob\checkpoint\assets
Epoch 2/11
4102/4102 [==============================] - 67s 16ms/step - loss: 0.4925 - accuracy: 0.8067 - val_loss: 0.4262 - val_accuracy: 0.8298
INFO:tensorflow:Assets written to: C:\Users\georg\Desktop\projects\Bob\checkpoint\assets
Epoch 3/11
4102/4102 [==============================] - 68s 17ms/step - loss: 0.4266 - accuracy: 0.8310 - val_loss: 0.3985 - val_accuracy: 0.8443
INFO:tensorflow:Assets written to: C:\Users\georg\Desktop\projects\Bob\checkpoint\assets
Epoch 4/11
4102/4102 [==============================] - 68s 17ms/step - loss: 0.3973 - accuracy: 0.8403 - val_loss: 0.3645 - val_accuracy: 0.8529
INFO:tensorflow:Assets written to: C:\Users\georg\Desktop\projects\Bob\checkpoint\assets
Epoch 5/11
4102/4102 [==============================] - 68s 

In [7]:
%load_ext tensorboard


In [8]:

%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 13880), started 0:14:15 ago. (Use '!kill 13880' to kill it.)

In [7]:
from tensorflow import keras
model = tf.keras.models.load_model('checkpoint')

In [7]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("type3.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\georg\AppData\Local\Temp\tmptyil6m1e\assets


49536

AttributeError: 'BatchDataset' object has no attribute 'shape'